## 1.13 ##

$\newcommand{\norm}[1]{\Vert #1 \Vert}$
$\newcommand{\intpi}[1]{\int_{-\pi}^{\pi} #1 dx}$

### a) ###
Here is the procedure for the first 2 basis vectors.

$e^{(0)}$:

$\intpi{x^0} = \intpi{1} = 2\pi$

So $e^{(0)} = \dfrac{1}{\sqrt{2\pi}}$

$e^{(1)}$:

$proj_{e^{(0)}}v^{(1)} = \dfrac{1}{\sqrt{2\pi}}\intpi{x}$

$u = v^{(1)} - proj_{e^{(0)}}v^{(1)}e^{(0)} = v^{(1)} - \dfrac{1}{2\pi}\intpi{x}$

$e^{(1)} = \dfrac{u}{\norm{u}}$

This is pretty repetitive afterwards, so I put it in code.

In [1]:
import scipy.integrate as integrate
import numpy as np

max_allowable_err = 1E-5

# Gets the value of the polynomial with the given coeffs, 
# c[0] * 1, ..., + c[n] * x ** n.
def get_poly(x, c):
    sum = 0.0
    for i in range(len(c)):
        sum += c[i] * x**i
    return sum

# We're always doing integration from -pi to pi.
def pi_integrate(f):
    result, max_err = integrate.quad(f, -np.pi, np.pi)
    err_message = 'Upper bound on integration error was: ' + str(max_err)
    assert max_err < max_allowable_err, err_message
    return result

# E.g coeffs[4] stores the 5 coefficients for the final basis vector computed 
# from Graham-Schmidt. 
coeffs = []

# The first basis vector has already been computed analytically.
coeffs.append([ 1.0 / np.sqrt(2 * np.pi) ])
max_power = 5

# Compute the bases e^(1), ..., e^(max_power).
for i in range(1, max_power + 1):
    
    # Get the projection coefficients.
    v = lambda x: x ** i
    proj_coeffs = []
    for j in range(0, i):
        basis = lambda x: get_poly(x, coeffs[j])
        proj_coeff = pi_integrate(lambda x: v(x) * basis(x))
        proj_coeffs.append(proj_coeff)
        
    # Aggregate coefficients for each polynomial.
    unnorm_coeffs = [0] * i
    for j in range(len(proj_coeffs)):
        for k in range(len(coeffs[j])):
            unnorm_coeffs[k] -= proj_coeffs[j] * coeffs[j][k]
    
    # Subtract the projections and get the norm.
    subtracted = lambda x: v(x) + get_poly(x, unnorm_coeffs)
    norm_func = lambda x: subtracted(x) * subtracted(x)
    norm = np.sqrt(np.abs(pi_integrate(norm_func)))
    coeffs.append([])
    for j in range(0, i):
        coeffs[i].append(unnorm_coeffs[j] / norm)
    coeffs[i].append(1.0 / norm)

# Test orthonormality.
for i in range(max_power + 1):
    for j in range(i, max_power + 1):
        f = lambda x: get_poly(x, coeffs[i]) * get_poly(x, coeffs[j])
        inner_prod = np.sqrt(np.abs(pi_integrate(f)))
        if i == j:
            assert np.abs(inner_prod - 1.0) < max_allowable_err
        else:
            assert np.abs(inner_prod) < max_allowable_err

The resulting coefficients vector $\alpha$ for each of $e^{(0)}, ..., e^{(5)}$ is printed below:

In [2]:
np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)
for i, alpha in enumerate(coeffs):
    print('The alpha(coefficients) vector for e^(%d):' % i)
    print(np.array(alpha))
    print('')

The alpha(coefficients) vector for e^(0):
[0.39894228]

The alpha(coefficients) vector for e^(1):
[0.         0.21994841]

The alpha(coefficients) vector for e^(2):
[-0.44603103  0.          0.13557718]

The alpha(coefficients) vector for e^(3):
[ 0.         -0.50396511  0.          0.0851039 ]

The alpha(coefficients) vector for e^(4):
[ 0.44881007  0.         -0.45473967  0.          0.05375389]

The alpha(coefficients) vector for e^(5):
[ 0.          0.78969213  0.         -0.37339186  0.          0.03404925]



### b) ###
To compute the $\alpha$ coefficients for $sin(x)$, we need to add up all the inner products of $sin(x)$ with our orthonormal basis.

In [89]:
# Compute projection coefficients.
basis_coeffs = []
for i in range(len(coeffs)):
    f = lambda x: np.sin(x) * get_poly(x, coeffs[i])
    basis_coeff = pi_integrate(f)
    basis_coeffs.append(basis_coeff)

# Aggregate.
alphas = [0] * (max_power + 1)
for i in range(len(basis_coeffs)):
    for j in range(len(coeffs[i])):
        alphas[j] += basis_coeffs[i] * coeffs[i][j]

print('Alphas(from a_0 to a_5):')
print(np.array(alphas))    

Alphas(from a_0 to a_5):
[ 0.          0.98786214  0.         -0.15527141  0.          0.00564312]


### c) ###
The even coefficients are all 0. This makes intuitive sense because $sin(x)$ is an odd function and should not have any non-zero even components, especially since it is vertically centered at $y = 0$.

### d) ###

The Taylor approximation is very good for the interval $ [-\dfrac{\pi}{2}, \dfrac{\pi}{2}]$, and starts to diverge outside of this interval.

The orthonormal polynomial basis projection is a better approximator for the function over the entire interval because we explicitly optimized for the pointwise squared difference. That is, it is important to approximate well at all points, and not just near the origin. 

The Taylor series approximates well at the origin as it is a _local_ descriptor and uses derivatives at the origin only. 

Plots below:

In [95]:
import plotly.offline as py
import plotly.graph_objs as go
py.offline.init_notebook_mode(connected=True)

N = 200
x = np.linspace(-np.pi, np.pi, N)
taylor_approx_f = lambda x: x - x ** 3 / 6.0 + x ** 5 / 120.0
taylor_approx = np.array(map(taylor_approx_f, x))
my_approx_f = lambda x: get_poly(x, alphas)
my_approx = np.array(map(my_approx_f, x))

taylor_approx_data = go.Scatter(x = x, y = taylor_approx, name='Taylor')
my_approx_data = go.Scatter(x = x, y = my_approx, name='Orthonormal')
py.iplot([taylor_approx_data, my_approx_data], filename='q1.13-d')